#Social data mining, Shilad Sen
#Assignment 1: 35 points

##Question 1: Modeling tweets

<style type="text/css">
iframe.twitter-tweet {
width:800px !important;
}
</style>

**Background:** In this question you will practice creating and navigating data structures that represent real world social media. At the end of this question you should be more comfortable translating social media into Python data structures. 

**Completion time:** This question should take you approximately one hour. 

**Point value:** Ungraded, immediate feedback.

**Part A.** Write Python code that declares a variable called `tweets` and fills it with three tweets.

 * The top level `tweets` data structure should be organized by username. Given a username, we should be able to easily retrieve the user's tweets.
 * Each tweet should contain three attributes: an author's username, the text of the tweet, and the usernames that have favorited the tweet. **You only need to include a few usernames that favorite each tweet.**
 * You may find it easier to fill the tweets variable in several steps, instead of defining it all at once.

<blockquote class="twitter-tweet" lang="en" width="100%"><p>News! Based on <a href="https://twitter.com/NateSilver538">@NateSilver538</a>&#39;s calculations, there&#39;s a 90.617854% chance we’re relaunching FiveThirtyEight on March 17.</p>&mdash; FiveThirtyEight (@FiveThirtyEight) <a href="https://twitter.com/FiveThirtyEight/statuses/442372247379247105">March 8, 2014</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

<blockquote class="twitter-tweet" lang="en"><p>The Fox knows many little things; The hedgehog knows one big thing. <a href="http://t.co/GWSmHaatPD">http://t.co/GWSmHaatPD</a> <a href="http://t.co/4s6rNpp5zi">http://t.co/4s6rNpp5zi</a></p>&mdash; FiveThirtyEight (@FiveThirtyEight) <a href="https://twitter.com/FiveThirtyEight/statuses/442384984540999680">March 8, 2014</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

<blockquote class="twitter-tweet" lang="en"><p>What scientific idea is ready for retirement? It is the standard deviation! (according to Nassim Taleb) <a href="http://t.co/0xvKRa1UNb">http://t.co/0xvKRa1UNb</a></p>&mdash; Jure Leskovec (@jure) <a href="https://twitter.com/jure/statuses/423611752417202176">January 16, 2014</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>


**Part B.** Write Python code to count and print out the total number of times each users' tweets have been favorited. As of March 9, 2014, this would be:

<pre>
    ('number of favorites for tweeter', 'FiveThirtyEight', 273)
    ('number of favorites for tweeter', 'Jure Leskovec', 13)
</pre>

In [16]:
import collections

user_tweets = collections.defaultdict(list) # this is a dictionary of lists.  Should return an empty list if we ash for a non-indexed value
print user_tweets['paul'] # should return an emptylist

# each item in our dictionay is a list of dictionaries.  the favorited dictionary item is a set.
user_tweets['FiveThirtyEight'].append({
           'userName' : 'FiveThirtyEight',
           'text' : 'News! Based on @NateSilver538\'s calculations, there\'s a 90.617854% chance we’re relaunching FiveThirtyEight on March 17',
           'favorited' : set(['TomSeleck', 'GraceJones','TommyTall'])
    })
user_tweets['FiveThirtyEight'].append({
           'userName' : 'FiveThirtyEight',
           'text' : 'The Fox knows many little things; The hedgehog knows one big thing.  http://www.fivethirtyeight.com/2014/03/fivethirtyeight-to-relaunch-on-march-17.html http://bit.ly/1ilaNwl',
           'favorited' : set(['BigBen', 'ThomasEnsley'])
    })

user_tweets['jure'].append({
           'userName' : 'jure',
           'text' : 'What scientific idea is ready for retirement? It is the standard deviation! (according to Nassim Taleb) http://www.edge.org/response-detail/25401 …',
           'favorited' : set(['Kit Carson', 'Davy_Jones','Michael the Archangel'])
    })

        
# loop over each user
for user in user_tweets:
    
    # accumulate sum over all tweets by that user    
    # alternately, with fancy list comprehensions: 
    # n = sum([len(tweet['favorited']) for tweet in user_tweets[user])
    n = 0
    for tweet in user_tweets[user]:
        n += len(tweet['favorited'])
        
    print('number of favorties for tweeter', user, n)



[]
('number of favorties for tweeter', 'jure', 3)
('number of favorties for tweeter', 'paul', 0)
('number of favorties for tweeter', 'FiveThirtyEight', 5)


## Question 2: Finding the highest rated movies.

**Background:** In this question you will practice dealing with noisy social data that has widely varying sample sizes. Upon completing this assignment, you will 1) Know how to estimate simple values like the mean in a way that accounts for small sample sizes. 2) Write programs that analyze data in a streaming structure. 3) Use dictionaries (and more specifically Python's defaultdict) to create an accumulator for different groups of samples. 

**Completion time:** This question should take you about two hours.

**Point value:** 10 points

**Instructions:** Write  a Python program that calculates the robust means for all movies in the MovieLens 10M dataset. Note: your program **must not** store all the ratings for each movie in memory. 

 1. Your program should create an accumulator variable called `sums` that keeps track of the sum of ratings for each movies.
 1. Your program should create an accumulator variable called `counts` that keeps track of the count of ratings for each movie.
 1. Your program should stream through each movie rating.
 1. For each rating, your program should update the two accumulator data structures.
 1. Your program should create a dictionary called `robust_means` whose keys will be movie ids and values will be robust means.
 1. Your program should fill the `robust_means` dictionary using data in the `sums` and `counts` accumulators.
 1. Your program should print the movie id, number of ratings, real mean, and robust mean for the 20 movies with highest and lowest robust means. Hint: You'll need to sort the robust_means dictionary by value. See [this Stackoverflow post](http://stackoverflow.com/questions/613183/python-sort-a-dictionary-by-value/3177911#3177911) for help.


In [81]:
path = 'C:/Users/PAUL/Downloads/ml-10m/ml-10M100K/ratings.dat'  # replace this with the full path to the ratings file on your computer
movieTitlePath = 'C:/Users/PAUL/Downloads/ml-10m/ml-10M100K/movies.dat'  # replace this with the full path to the ratings file on your computer

import collections

# accumulator histogram of raw counts
histogram = collections.defaultdict(int)

# accumulators necessary for computing mean vaues of each movie.
sums = collections.defaultdict(int)    #keeps track of the sum of ratings for each movies
counts = collections.defaultdict(float)  #keeps track of the count of ratings for each movie
robustMeans = {} # dictionray to hold computed robust means for each, key = movieID
realMeans = {} # dictionray to hold computed robust means for each, key = movieID
numFakeRatings=5

# read in movie titles
movieTitles = collections.defaultdict(str)
for line in open('C:/Users/PAUL/Downloads/ml-10m/ml-10M100K/movies.dat'):
    tokens = line.split('::')    # tokens is a list of the four values in each recor
    movieID = int(tokens[0])
    movieTitles[movieID] = tokens[1]

totalMovieRatingSum=0.0
totalMovieRatingCount=0

fp = open(path,"r")
for line in fp:
    tokens = line.split('::')    # tokens is a list of the four values in each recor
    
    # accumulate individual movie ratings and counts
    movieID = int(tokens[1])
    movieRatingValue =  float(tokens[2])
    
    counts[movieID] +=1
    sums[movieID] += movieRatingValue
    
    # accumulate total movie ratings and counts to compute the grand average for all movies
    # to use a the value for our fake ratings.
    totalMovieRatingSum+=movieRatingValue
    totalMovieRatingCount+=1
    

# compute the value for our fake rating padding
fakeRating = totalMovieRatingSum/totalMovieRatingCount

#compute robustMeans and actual for all movies
for movieID in counts.keys():
    realMeans[movieID]= sums[movieID]/counts[movieID]
    robustMeans[movieID]= (sums[movieID]+(numFakeRatings*fakeRating))/(counts[movieID]+numFakeRatings)

  
fp.close()

print "Top 20 Movies by Sorted by Robust Mean"
for w in (sorted(robustMeans, key=robustMeans.get, reverse=True))[0:20]:
  print "movie_id = " + str(w) + " " + movieTitles[w], "num movie ratings = " + str(counts[w]),  "robust_mean = " + str(robustMeans[w]), "real mean = " + str(realMeans[w]) 
print "\n"

print "Bottom 20 Movies by Sorted by Robust Mean"
for w in (sorted(robustMeans, key=robustMeans.get, reverse=False))[0:20]:
  print "movie_id = " + str(w) + " " + movieTitles[w], "num movie ratings = " + str(counts[w]),  "robust_mean = " + str(robustMeans[w]), "real mean = " + str(realMeans[w]) 


Top 20 Movies by Sorted by Robust Mean
movie_id = 318 Shawshank Redemption, The (1994) num movie ratings = 31126.0 robust_mean = 4.45708657318 real mean = 4.45723832166
movie_id = 858 Godfather, The (1972) num movie ratings = 19814.0 robust_mean = 4.41485756646 real mean = 4.41508529323
movie_id = 50 Usual Suspects, The (1995) num movie ratings = 24037.0 robust_mean = 4.36696456658 real mean = 4.36714232225
movie_id = 527 Schindler's List (1993) num movie ratings = 25777.0 robust_mean = 4.36331790046 real mean = 4.36348294992
movie_id = 922 Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) num movie ratings = 3255.0 robust_mean = 4.32072457352 real mean = 4.32196620584
movie_id = 912 Casablanca (1942) num movie ratings = 12507.0 robust_mean = 4.31941832718 real mean = 4.31974094507
movie_id = 904 Rear Window (1954) num movie ratings = 8825.0 robust_mean = 4.31608857414 real mean = 4.31654390935
movie_id = 3435 Double Indemnity (1944) num movie ratings = 2403.0 robust_mean = 4.31377164023 r

## Question 3: If you liked "Star Wars," you'll like ?

**Introduction:** For this question, you will generalize the code above to simultaneously calculate the correlation with Star Wars and every other movie. Upon completion of this homework, 1) You'll know how to build an associative recommender system, and 2) You'll have gained experience in using nested accumulator data structures for streaming datasets.

**Completion time:** This question should require 3-5 hours.

**Point value:** 15 points

**Generalizing the correlation algorithm:**
Recall the structure of our cosimilarity code above. 
To calculate the correlation between Star Wars and Finding Nemo, we needed two lists: X and Y.
Each list contained one entry for every user who rated both movies.
For example, X[2] contains the rating for Toy Story for the third user who rated both movies, and Y[2] contains the same user's rating for Finding Nemo.

To generalize this program to simultaneously calculate the correlation between ratings for Star Wars and every other movie, we need to simultaneously calculate the X and Y lists for every other movie. We'll use one data structure that calculates every movie's X called `every_X` and a similar datastructure called `every_Y`. These will be nested datastructures? What will they look like?

Note that although `every_X` always contains user ratings for Star Wars, it will contain a different list of ratings for each candidate movie (i.e. the lists stored by `every_X` will be different for Lion King and Finding Nemo).
This is because each list in `every_X` only includes ratings for users who rated both Star Wars and the candidate movie.

**Helper function for reading movie titles:** Your program will need to look up a movie's title given its id. The read_titles method below returns a dictionary whose keys are movie ids (ints) and values are titles:

In [28]:
def read_titles(path):
    """
        Read in the movie ids and their associated titles.
        Returns a hashtable containing the association.
        Note that the ids are ints.  
    """
    titles = {}
    for line in open(path, 'r'):
        tokens = line.split('::')
        if len(tokens) >= 2:
            (id, title) = tokens[:2]
            titles[int(id)] = title
    return titles

# replace this with the full path to the movies.dat on your computer
path_movies = '/Users/shilad/Downloads/ml-10M100K/movies.dat' 

titles = read_titles(path_movies)
for i in range(1, 10):
    print(i, titles[i])

IOError: [Errno 2] No such file or directory: '/Users/shilad/Downloads/ml-10M100K/movies.dat'

**Instructions:**

 1. Create two data structures to hold rating lists for every movie: `every_X` and `every_Y`. 
    Carefully plan the contents of these data structures and describe them in your source code.
 1. While reading in users via the `read_users()` method, skip all users who have not rated Star Wars (movie id 260).
 1. For each user who has rated Star Wars, update `every_X` and `every_Y` for each movie they have rated.
 1. After you have finished processing all users, create a dictionary called `every_correlation`.
 1. Fill `every_correlation` with the correlations for every movie with at least 1000 ratings using the data in `every_X` and `every_Y`.
 1. Print the 20 movies along with highest correlations. You'll need to sort the dictionary by values, as you did in Assignment 1.2.  The first two movies you should see should be obvious choices for the movies most related to Star Wars.  If they aren't, you've done something wrong.

In [137]:
from scipy.stats import pearsonr
def read_titles(movieDataPath):
    """
        Read in the movie ids and their associated titles.
        Returns a hashtable containing the association.
        Note that the ids are ints.  
    """
    titles = {}
    for line in open(movieDataPath, 'r'):
        tokens = line.split('::')
        if len(tokens) >= 2:
            (id, title) = tokens[:2]
            titles[int(id)] = title
    return titles



def read_users(movieRatingspath):
    """
        Generator function used to iterate over large file of 
        movie ratings.  File is assumed sorted by user
        Generator will return dictionary of all user ratings
    """

    user_ratings = {}     # dictionary from movie id to user rating
    prev_user_id = -1     #  user id
    
    for line in open(movieRatingspath):
        tokens = line.split('::')    # tokens is a list of the four values in each record
        user_id = int(tokens[0])
        movie_id = int(tokens[1])
        rating = float(tokens[2])

            
        # if the current line represents a new user, process the previous user's ratings
        if user_id != prev_user_id and len(user_ratings) > 0:
            # this tells Python to use the value of user_ratings 
            # for the next iteration of the for loop at the bottom of this cell
            #print "user_id = " + str(user_id), "prev_user_id" + str(prev_user_id)
            yield user_ratings
            
            # mark new user as current user, clear out old user's ratings
            user_ratings.clear()
            prev_user_id = user_id        
        
        # record user rating
        user_ratings[movie_id] = rating
        prev_user_id = user_id
    
    # process the very last user in the file
    #print "user_id = " + str(user_id), "prev_user_id" + str(prev_user_id)
    yield user_ratings

path = 'C:/Users/PAUL/Downloads/ml-10m/ml-10M100K/ratings.dat'  # replace this with the full path to the ratings file on your computer
movieTitlePath = 'C:/Users/PAUL/Downloads/ml-10m/ml-10M100K/movies.dat'  # replace this with the full path to the ratings file on your computer


every_X=collections.defaultdict(list)  # every_X will be a dictionary indexed by movie id.  Each dictionary item will be a list.  Each Item in the list
           # will be a users movie rating for star wars
every_Y=collections.defaultdict(list)  # every_Y will be a dictionary indexed by movie id.  Each dictionary item will be a list.  Each Item in the list
           # will be a users movie rating for the indexed movie.

every_correlation = {} # dictionary to hold movie correlations.  Indexed by movie id.  Each entry is the pearson correlation coefficient as a float.

num_users = 0
for user in read_users(path):
    num_users += 1

    #print user
    
    #determine if this user rated star wars.  movie id for start wars is 260
    if 260 in user.keys():
        #print 'Found target'
        # get Star Wars Rating
        starWarsRating=user[260]
        for movieID in user.keys():
            every_X[movieID].append(starWarsRating)
            every_Y[movieID].append(user[movieID])
        


        
"""        
print('num_users:', num_users)
print every_X
print every_Y
"""
for movieID in every_X.keys():
    #print movieID
    if len(every_X[movieID]) > 1000:
        #print "movieID = " +str(movieID), " is length " + str(len(every_X[movieID]))
        #print  every_X[movieID]
        #print  every_Y[movieID]
        r_s, p = pearsonr(every_X[movieID], every_Y[movieID])
        #print r_s, p
        every_correlation[movieID] = r_s
print len(every_correlation)


movieTitles = read_titles(movieTitlePath)

print "Top 20 Movies You Might Like if You Like Star Wars"
for w in (sorted(every_correlation, key=every_correlation.get, reverse=True))[0:20]:
  print "movie_id = " + str(w) + " " + movieTitles[w], " pearson correlation = " + str(every_correlation[w]) + " number of reviews = " + str(len(every_X[w]))
print "\n"

print "Top 20 Movies You Won't Like if You Like Star Wars"
for w in (sorted(every_correlation, key=every_correlation.get, reverse=False))[0:20]:
  print "movie_id = " + str(w) + " " + movieTitles[w], " pearson correlation = " + str(every_correlation[w]) + " number of reviews = " + str(len(every_X[w]))
print "\n"

1659
Top 20 Movies You Might Like if You Like Star Wars
movie_id = 260 Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977)  pearson correlation = 1.0 number of reviews = 28566
movie_id = 1196 Star Wars: Episode V - The Empire Strikes Back (1980)  pearson correlation = 0.721649903836 number of reviews = 19572
movie_id = 1210 Star Wars: Episode VI - Return of the Jedi (1983)  pearson correlation = 0.663125010211 number of reviews = 20649
movie_id = 1198 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)  pearson correlation = 0.462407332137 number of reviews = 17006
movie_id = 33493 Star Wars: Episode III - Revenge of the Sith (2005)  pearson correlation = 0.414449461842 number of reviews = 4643
movie_id = 2628 Star Wars: Episode I - The Phantom Menace (1999)  pearson correlation = 0.402800058672 number of reviews = 12491
movie_id = 5378 Star Wars: Episode II - Attack of the Clones (2002)  pearson correlation = 0.394955311688 number of reviews = 6599


## Question 4: Analyzing time complexity

**Introduction:** For this question, you will practice analyzing the time complexity of Python code. Although understanding time complexity well requires a dedicated course, when you finish this question you'll improve your intution about time complexity.

**Completion time:** This question should require about 2 hours.

**Point value:** 10 points.

**Part A:** I've given you three Python functions below: f, g, and h. Each has a different time complexity. If you run the cell below in inotebook, it will report how long it takes to run f, g, and h. 

In [136]:
def f(n):
    n += 1
    return

def g(n):
    j = 1
    for i in range(n):
        j += 1
    return

def h(n):
    j = 1
    for i in range(n):
        for k in range(n):
            j += 1
    return

n = 1000
print "n = 1000"
%time f(n)
%time g(n)
%time h(n)
print "\n"

n = 2000
print "n = 2000"
%time f(n)
%time g(n)
%time h(n)
print "\n"

n = 4000
print "n = 4000"
%time f(n)
%time g(n)
%time h(n)
print "\n"

n = 8000
print "n = 8000"
%time f(n)
%time g(n)
%time h(n)
print "\n"

n = 16000
print "n = 16000"
%time f(n)
%time g(n)
%time h(n)
print "\n"

n = 32000
print "n = 32000"
%time f(n)
%time g(n)
%time h(n)

n = 64000
print "n = 64000"
%time f(n)
%time g(n)
%time h(n)


n = 128000
print "n = 128000"
%time f(n)
%time g(n)
%time h(n)

n = 1000
Wall time: 0 ns
Wall time: 0 ns
Wall time: 63 ms


n = 2000
Wall time: 0 ns
Wall time: 0 ns
Wall time: 265 ms


n = 4000
Wall time: 0 ns
Wall time: 0 ns
Wall time: 1.09 s


n = 8000
Wall time: 0 ns
Wall time: 0 ns
Wall time: 4.08 s


n = 16000
Wall time: 0 ns
Wall time: 2 ms
Wall time: 15.3 s


n = 32000
Wall time: 0 ns
Wall time: 2 ms
Wall time: 1min 2s
n = 64000
Wall time: 0 ns
Wall time: 5 ms
Wall time: 6min 16s
n = 128000
Wall time: 0 ns
Wall time: 10 ms
Wall time: 32min 18s


Time how long it takes f, g, and h to run as you increase n by factors of 2 (n=1000, 2000, 4000, etc.). Then do the following:

 1. Record  (via markdown or comments) how long it takes to run f, g, and h for each value of n.
 1. Describe the patterns do you see for each.
 1. What time complexities do your observations suggest for f, g, and h?
 1. Why does the code in each function generate the time complexities you observed?

<h4> 1 Record (via markdown or comments) how long it takes to run f, g, and h for each value of n. </h4>

<table>
<tr>
<th>n</th><th>run time f</th><th>run time g</th><th>run time h</th>
</tr>

<tr>
<td>1,000</td> <td>0</td>  <td>0</td>  <td>63</td>
</tr>

<tr>
<td>2,000</td> <td>0</td>  <td>0</td>  <td>256</td>
</tr>

<tr>
<td>4,000</td> <td>0</td>  <td>0</td>  <td>1,090</td>
</tr>

<tr>
<td>8,000</td> <td>0</td>  <td>0</td>  <td>4,080</td>
</tr>

<tr>
<td>16,000</td> <td>0</td>  <td>2</td>  <td>15,600</td>
</tr>

<tr>
<td>32,000</td> <td>0</td>  <td>2</td>  <td>62,000</td>
</tr>

<tr>
<td>64,000</td> <td>0</td>  <td>5</td>  <td>37,800</td>
</tr>

<tr>
<td>64,000</td> <td>0</td>  <td>10</td>  <td> 1,921,800.00 
</td>
</tr>
</table>

<h4>2. Describe the patterns do you see for each.</h4>
<p><b>- run time f</b> The run time of function f is so quick it is not measurable and doesn't increase with size of (n) </p>
<p><b>- run time g</b> The run time of function g increases linearly with size of (n) </p>
<p><b>- run time h</b> The run time of function h increases faster than linearly with size of (n) </p>

<h4>3. What time complexities do your observations suggest for f, g, and h?</h4>
<p><b>- time complexity f = </b>  O(1) </p>
<p><b>- time complexity g = </b> O(n)</p>
<p><b>- time complexity h = </b> O(n^2)</p>

<h4>Why does the code in each function generate the time complexities you observed?</h4>
<p>Function f does not iterate over the data so its run time will not depend on the size of n</p>
<p>Function g iterates just once over the data so its run time will be linearly proporational to the size of n </p>
<p>Function h has nested for loops so it will run proportionally to the square of the size of </p>

**Part B:** Repeat the previous procedure from Part 1 for the functions p and q below:

In [139]:
def p(n):
    l = []
    for i in range(n):
        l.insert(0, 'foo')     # insert 'foo' to the beginning (the 0'th position)
def q(n):
    l = []
    for i in range(n):
        l.append('foo')        # append 'foo' to the end


n = 10000
print "n = 10,000"
%time p(n)
%time q(n)
print "\n"
        
n = 20000
print "n = 20,000"
%time p(n)
%time q(n)
print "\n"       
        
n = 40000
print "n = 40,000"
%time p(n)
%time q(n)
print "\n"

print 
n = 80000
print "n = 80,000"
%time p(n)
%time q(n)
print "\n"

n = 160000
print "n = 160,000"
%time p(n)
%time q(n)
print "\n"

n = 320000
print "n = 320,000"
%time p(n)
%time q(n)
print "\n"

n = 10,000
Wall time: 58 ms
Wall time: 2 ms


n = 20,000
Wall time: 221 ms
Wall time: 3 ms


n = 40,000
Wall time: 845 ms
Wall time: 6 ms



n = 80,000
Wall time: 3.4 s
Wall time: 13 ms


n = 160,000
Wall time: 13.6 s
Wall time: 26 ms


n = 320,000
Wall time: 54.8 s
Wall time: 56 ms




Time how long it takes p and q to run as you increase n by factors of 2 (this time start with n=10000). Then do the following:
 
 <h4>1. Record  (via markdown or comments) how long it takes to run p and q for each value of n.</h4>
 
<table>
<tr>
<th>n</th><th>run time p</th><th>run time q</th>
</tr>

<tr>
<td>10,000</td> <td>58</td>  <td>2</td> 
</tr>

<tr>
<td>20,000</td> <td>221</td>  <td>3</td>  
</tr>

<tr>
<td>40,000</td> <td>845</td>  <td>6</td> 
</tr>

<tr>
<td>80,000</td> <td>3,400</td>  <td>13</td> 
</tr>

<tr>
<td>160,000</td> <td>13,600</td>  <td>26</td> 
</tr>

<tr>
<td>320,000</td> <td>54,800</td>  <td>56</td> 
</tr>


</table>
 
<h4> 2. Describe the patterns do you see for each.</h4>
<p>Function p is increasing close to proportionately to n-sqaured.  I put this data into excel and used the regression function with a power function specification and it almost perfectly fit this equation. y = 7E-07x^1.9788 R² = 0.9999 <p>
<p>Function q is increasing at a linear rate.  Again, using Excel's regression function and specifying a linear model gives these results.  y = 0.0002x - 0.7065R² = 0.9985</p>


<h4> 3. What time complexities do your observations suggest for p and q? </h4>
<p>Function p has a time complexity of O(n^2>.  Function q has a time complexity of O(n)</p>


<h4> 4. Why does the code in each function generate the time complexities you observed? Hint: Python's list uses an array that stores elements in consecutive memory locations. </h4>

<p>Function q has a linear complexity because it simply needs to to traverse from the head of the list to the end and then append a new element to the end of the list.  Function p has a quadratic complexity because it must create an entire new data list data structure since the elements must fit in contiquous memory elements</p>

**Part C**: Estimate the time complexity of the code you wrote for question 2 in terms of the number of movie ratings n. You do not need to include an analysis of step 7 - only steps 1 through 6. Explain your answer.

To help you, consider what actions you perform for each rating. What data structures are used? How fast are the operations you use on those data structures?
 
 1. Your program should create an accumulator variable called `sums` that keeps track of the sum of ratings for each movies.  <b>=complexity O(1)</b>
 1. Your program should create an accumulator variable called `counts` that keeps track of the count of ratings for each movie.<b>=complexity O(1)</b>
 1. Your program should stream through each movie rating.<b>=complexity O(n)</b>
 1. For each rating, your program should update the two accumulator data structures.=complexity O(n)</b>
 1. Your program should create a dictionary called `robust_means` whose keys will be movie ids and values will be robust means.<b>=complexity O(1)</b>
 1. Your program should fill the `robust_means` dictionary using data in the `sums` and `counts` accumulators.<b>=complexity O(n)</b>
 
<p><b>Overall the first 6 steps of this program would have a linear time complexity proportional to the number of movie ratings n, this would be O(n).</b></p>



**Part D**: Estimate the time complexity of the code you wrote for question 3 in terms of the number of movie ratings n. You do not need to include an analysis of step 6 - only steps 1 through 5. 

To make your analysis simpler, presume *every* user has rated Star Wars. How would you adjust your analysis to account for the fact that only some people rate Star Wars?

Make sure to explain your answer.

As a hint, consider what actions you perform for each rating. What data structures are used? How fast are the operations you use on those data structures?

 1. Create two data structures to hold rating lists for every movie: `every_X` and `every_Y`. <b>=complexity O(1)</b>
    Carefully plan the contents of these data structures and describe them in your source code.
 1. While reading in users via the `read_users()` method, skip all users who have not rated Star Wars (movie id 260).<b>=complexity O(n) </b>
 1. For each user who has rated Star Wars, update `every_X` and `every_Y` for each movie they have rated.<b>=complexity O(n)</b>
 1. After you have finished processing all users, create a dictionary called `every_correlation`.<b>=complexity O(1)</b>
 1. Fill `every_correlation` with the correlations for every movie with at least 1000 ratings using the data in `every_X` and `every_Y`.<b>=complexity O(n)</b>

<b>Overall the first 5 steps of this program would have a quadratic time complexity proportional to the number of users u times  movie ratings n.  Because step 3 is nested in side step 2,  this would be O(n^2). </b>